In [0]:
! git clone https://github.com/Nikronic/ObjectNet.git

Cloning into 'ObjectNet'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 141 (delta 1), reused 17 (delta 0), pack-reused 123
Receiving objects: 100% (141/141), 2.01 MiB | 5.17 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [0]:
! git clone https://github.com/Nikronic/CoarseNet.git

Cloning into 'CoarseNet'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 210 (delta 3), reused 6 (delta 0), pack-reused 198
Receiving objects: 100% (210/210), 177.46 KiB | 1.42 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [0]:
! git clone https://github.com/Nikronic/EdgeNet.git

Cloning into 'EdgeNet'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 98 (delta 54), reused 63 (delta 22), pack-reused 0
Unpacking objects: 100% (98/98), done.


In [0]:
% cd ObjectNet/

/content/ObjectNet


In [0]:
%%bash
pwd
rm -r ObjectNet/
rm -r EdgeNet/
rm -r sample_data/

In [0]:
pwd

'/content/ObjectNet'

### Download pretrained weights of ObjectNet

In [0]:
%%bash

# Image and model names
TEST_IMG=ADE_val_00001519.jpg
MODEL_PATH=baseline-resnet101dilated-ppm_deepsup
RESULT_PATH=./

ENCODER=$MODEL_PATH/encoder_epoch_25.pth
DECODER=$MODEL_PATH/decoder_epoch_25.pth

# Download model weights and image
if [ ! -e $MODEL_PATH ]; then
  mkdir $MODEL_PATH
fi
if [ ! -e $ENCODER ]; then
  wget -P $MODEL_PATH http://sceneparsing.csail.mit.edu/model/pytorch/$ENCODER
fi
if [ ! -e $DECODER ]; then
  wget -P $MODEL_PATH http://sceneparsing.csail.mit.edu/model/pytorch/$DECODER
fi
if [ ! -e $TEST_IMG ]; then
  wget -P $RESULT_PATH http://sceneparsing.csail.mit.edu/data/ADEChallengeData2016/images/validation/$TEST_IMG
fi

# Inference
python3 -u test.py \
  --model_path $MODEL_PATH \
  --test_imgs $TEST_IMG \
  --suffix '_epoch_25.pth' \
  --arch_encoder resnet101dilated \
  --arch_decoder ppm_deepsup \
  --fc_dim 2048 \
  --result $RESULT_PATH


Input arguments:
test_imgs        ['ADE_val_00001519.jpg']
model_path       baseline-resnet101dilated-ppm_deepsup
suffix           _epoch_25.pth
arch_encoder     resnet101dilated
arch_decoder     ppm_deepsup
fc_dim           2048
num_val          -1
num_class        150
batch_size       1
imgSize          [300, 400, 500, 600]
imgMaxSize       1000
padding_constant 8
segm_downsampling_rate 8
result           ./
gpu              0
Loading weights for net_encoder
Loading weights for net_decoder
# samples: 1
Inference done!


100%|██████████| 1/1 [00:01<00:00,  1.36s/it]


# TRAIN.py  - - - - - - - - - - - - - - - EDGE NET

In [0]:
%%writefile train_edited.py

# %% import library
from EdgeNet import EdgeNet
from torchvision.transforms import Compose, ToPILImage, ToTensor, RandomResizedCrop, RandomRotation, \
    RandomHorizontalFlip
from EdgeNet.utils.preprocess import *
import torch
from torch.utils.data import DataLoader
from EdgeNet.utils.loss import EdgeLoss

import torch.optim as optim
import torch.nn as nn
from torch.backends import cudnn


###########################
# System libs
import os
import argparse
from distutils.version import LooseVersion
# Numerical libs
import numpy as np
import torch
import torch.nn as nn
from scipy.io import loadmat
# Our libs

from models import ModelBuilder, SegmentationModule
from utils import colorEncode
from lib.nn import user_scattered_collate, async_copy_to
from lib.utils import as_numpy
import lib.utils.data as torchdata
import cv2
from tqdm import tqdm

torch.utils
# import argparse

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# %% config parser
# parser = argparse.ArgumentParser()
# parser.add_argument("--txt", help='path to the text file', default='filelist.txt')
# parser.add_argument("--img", help='path to the images tar archive (uncompressed) or extracted folder', default='data')
# parser.add_argument("--txt_t", help='path to the text file of test set', default='filelist.txt')
# parser.add_argument("--img_t", help='path to the images tar archive (uncompressed) or extracted folder of test set',
#                     default='data')
# parser.add_argument("--bs", help='int number as batch size', default=128, type=int)
# parser.add_argument("--es", help='int number as number of epochs', default=10, type=int)
# parser.add_argument("--nw", help='number of workers (1 to 8 recommended)', default=4, type=int)
# parser.add_argument("--lr", help='learning rate of optimizer (=0.0001)', default=0.0001, type=float)
# parser.add_argument("--cudnn", help='enable(1) cudnn.benchmark or not(0)', default=0, type=int)
# parser.add_argument("--pm", help='enable(1) pin_memory or not(0)', default=0, type=int)
# args = parser.parse_args()

# cudnn.benchmark = False
# pin_memory = False
# cudnn.benchmark = True if args.cudnn == 1 else False
# pin_memory = True if args.cudnn == 1 else False


# %% define data sets and their loaders
custom_transforms = Compose([
    RandomResizedCrop(size=224, scale=(0.8, 1.2)),
    RandomRotation(degrees=(-30, 30)),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    RandomNoise(p=0.5, mean=0, std=0.1)])

train_dataset = PlacesDataset(txt_path='EdgeNet/filelist.txt',
                              img_dir='EdgeNet/data',
                              transform=custom_transforms)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=6,
                          pin_memory= False)

test_dataset = PlacesDataset(txt_path='EdgeNet/filelist.txt',
                             img_dir='EdgeNet/data',
                             transform=ToTensor())

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=128,
                         shuffle=False,
                         num_workers=0,
                         pin_memory=False)


# %% initialize network, loss and optimizer
def init_weights(m):
    """
    Initialize weights of layers using Kaiming Normal (He et al.) as argument of "Apply" function of
    "nn.Module"

    :param m: Layer to initialize
    :return: None
    """

    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_in')
        m.bias.data.fill_(0.0)
    elif isinstance(m, nn.BatchNorm2d):  # reference: https://github.com/pytorch/pytorch/issues/12259
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)


# %% train model
def train_model(net, data_loader, optimizer, criterion, epochs=10):
    """
    Train model

    :param net: Parameters of defined neural network
    :param data_loader: A data loader object defined on train data set
    :param epochs: Number of epochs to train model
    :param optimizer: Optimizer to train network
    :param criterion: The loss function to minimize by optimizer
    :return: None
    """

    edgenet = net[0]
    objectnet = net[1]
    
    edgenet.train()
    objectnet.eval()

    for epoch in range(epochs):

        running_loss = 0.0
        for i, data in enumerate(data_loader, 0):
            # get the inputs
            y_descreen = data['y_descreen']
            y_e = data['y_edge']

            y_descreen = y_descreen.to(device)
            y_e = y_e.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = edgenet(y_descreen)  # output of model to feed to concatnation
            
            outputs_obj = objectnet(y_descreen)
            
            loss = criterion(outputs, y_e.float())
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss))
            running_loss = 0.0
    print('Finished Training')


# %% test
def test_model(net, data_loader):
    """
    Return loss on test

    :param net: The trained NN network
    :param data_loader: Data loader containing test set
    :return: Print loss value over test set in console
    """

    edgenet = net[0]
    objectnet = net[1]
    
    edgenet.train()
    objectnet.eval()
    
    running_loss = 0.0
    with torch.no_grad():
        for data in data_loader:
            y_descreen = data['y_descreen']
            y_e = data['y_edge']

            y_descreen = y_descreen.to(device)
            y_e = y_e.to(device)
            outputs = edgenet(y_descreen)
            outputs_obj = objectnet(y_descreen) #####
            loss = criterion(outputs, y_e)
            running_loss += loss
            print('loss: %.3f' % running_loss)
    return outputs, outputs_obj


def show_test(image_batch):
    """
    Show a sample grid image which contains some sample of test set result

    :param image_batch: The output batch of test set
    :return: PIL image of all images of the input batch
    """

    to_pil = ToPILImage()
    fs = []
    for i in range(len(image_batch)):
        img = to_pil(image_batch[i].cpu())
        fs.append(img)
    x, y = fs[0].size
    ncol = 3
    nrow = 3
    cvs = Image.new('RGB', (x * ncol, y * nrow))
    for i in range(len(fs)):
        px, py = x * int(i / nrow), y * (i % nrow)
        cvs.paste((fs[i]), (px, py))
    cvs.save('out.png', format='png')
    cvs.show()


# %% run model
criterion = EdgeLoss()
edgenet = EdgeNet.EdgeNet().to(device)

builder = ModelBuilder()
net_encoder = builder.build_encoder(
    arch='resnet101dilated',
    fc_dim=2048,
    weights=os.path.join('baseline-resnet101dilated-ppm_deepsup', 'encoder' + '_epoch_25.pth'))
net_decoder = builder.build_decoder(
    arch='ppm_deepsup',
    fc_dim=2048,
    num_class=150,
    weights=os.path.join('baseline-resnet101dilated-ppm_deepsup', 'decoder' + '_epoch_25.pth'),
    use_softmax=True)

    

segmentation_module = SegmentationModule(net_encoder, net_decoder, criterion)
segmentation_module.cuda()

optimizer = optim.Adam(edgenet.parameters(), lr=0.0001)
edgenet.apply(init_weights)
train_model([edgenet, segmentation_module], train_loader, optimizer, criterion, epochs=20)
show_test(test_model(edgenet, test_loader)[0])
show_test(test_model(edgenet, test_loader)[1])




Overwriting train_edited.py


In [0]:
!python train_edited.py

Loading weights for net_encoder
Loading weights for net_decoder
Traceback (most recent call last):
  File "train_edited.py", line 238, in <module>
    train_model([edgenet, segmentation_module], train_loader, optimizer, criterion, epochs=20)
  File "train_edited.py", line 145, in train_model
    outputs_obj = objectnet(y_descreen)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 489, in __call__
    result = self.forward(*input, **kwargs)
  File "/content/ObjectNet/models/models.py", line 35, in forward
    pred = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True))
IndexError: too many indices for tensor of dimension 4


# Make sure using PYTORCH=1.0.x to run this code!

In [0]:
%%bash
pip uninstall pytorch torchvision -y
pip uninstall torch -y
pip uninstall torch -y  # yes twice

pip install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
pip install torchvision

cd semantic-segmentation-pytorch/
chmod +x demo_test.sh
./demo_test.sh
cd ..

In [0]:
import torch
print(torch.__version__, 'Do not consider this number. Python is telling bullshit!')
import sys
print(sys.version)

1.1.0 Do not consider this number. Python is telling bullshit!
3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
